**Práctica 2: Lectura y manipulación de datos con pandas**

En esta práctica, se cargan los datos en marcos de datos (inglés: data frame) con la librería pandas

https://pandas.pydata.org/

para poder iniciar su procesamiento. 

En el caso de estudio ejemplo de la profesora, las metas de la práctica son

1. combinar la información de los resultados de 1ra y 2da oportunidad con los datos de las encuestas
2. agregar una variable categórica para cada alumno, nombrando los casos de la siguiente forma
    * 1ra: aprobó la unidad de aprendizaje en primera oportunidad
    * 2da: aprobó la unidad de aprendizaje en segunda oportunidad
    * 3ra: presentó y reprobó ambas la 1ra y la 2da oportunidad
    * 5ta: no participó lo suficiente para tener derecho a segunda oportunidad
    *  NA: no se cuenta con información sobre sí o no participó o tuvo derecho en 1ra y 2da oportunidad 
3. extraer datos de encuestas para un caso específico o para una combinación de casos
4. determinar cuántos estudiantes por semestre y por maestro hay en cada caso

y cada quien propondrá metas de este tipo para su segundo reporte. 

El primer intento de 

In [ ]:
import pandas as pd
enc_ini = pd.read_csv("ini.csv")

lanza una multitud de errores:

In [ ]:
Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/io/parsers.py", line 678, in parser_f
    return _read(filepath_or_buffer, kwds)
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/io/parsers.py", line 446, in _read
    data = parser.read(nrows)
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/io/parsers.py", line 1036, in read
    ret = self._engine.read(nrows)
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/io/parsers.py", line 1848, in read
    data = self._reader.read(nrows)
  File "pandas/_libs/parsers.pyx", line 876, in pandas._libs.parsers.TextReader.read
  File "pandas/_libs/parsers.pyx", line 891, in pandas._libs.parsers.TextReader._read_low_memory
  File "pandas/_libs/parsers.pyx", line 945, in pandas._libs.parsers.TextReader._read_rows
  File "pandas/_libs/parsers.pyx", line 932, in pandas._libs.parsers.TextReader._tokenize_rows
  File "pandas/_libs/parsers.pyx", line 2112, in pandas._libs.parsers.raise_parser_error
pandas.errors.ParserError: Error tokenizing data. C error: Expected 19 fields in line 680, saw 35

Pinta que el número de columnas por renglón es variable. Investigemos. 

Abriendo el archivo en emacs, lo primero que resalta son los mugres ˆM saltos de línea de Windows, 
cortesía de Google Drive (por razones misterio). 

In [ ]:
emacs ini.csv 
mv ini.csv ini.win
mv mc.csv mc.win
mv ord.csv  ord.win
tr -d '\015' < ini.win > ini.csv
tr -d '\015' < mc.win > mc.csv
tr -d '\015' < ord.win > ord.csv

ya quita esa bronca. Ahora, con awk se puede checar lo de cuántos campos hay en cada renglón.

In [ ]:
awk -F "," '{print NF}' ini.csv | sort | uniq -c
 513 19
 149 20
  91 21
  59 22
  24 23
  14 24
   8 25
   9 26
   4 27
   1 28
   2 29
   1 30
   2 31
   1 38

inmediatamente da la idea que algo está muy mal ya que deberían ser las mismas preguntas en cada respuesta.
Vamos a ver cuáles líneas son las que faltan preguntas:

In [ ]:
awk -F "," '{if (NF < 17) {print $0} }' ini.csv 

y salen cosas como ésta:
    
...NA,"la hora clase y cuando de plano no le entienda o tenga tarea, en mis horas libres",juntarme con compañeros resolviendo dudas,70-79,En que alomejor batallo para aprender algunos conceptos basicos ya que nunca he llevado programacion,23,NA,no se,NA...

Esto pasa mucho con encuestas: al responder, han introducido commas en sus respuestas de texto,
y ahora pandas cambia de columna con cada comma...

Esto es rescatable, ya que cada respuesta que contiene commas está entre comillas dobles. 
El chiste es ahora recorrer todo el archivo y quitar las commas que vienen entre comillas dobles.

Usaré python para ilustrar el principio: 
* se lee el texto caracter por caracter
* al ver una comilla doble, se levanta una bandera (variable booleana)
* cada comma que viene mientras la bandera está arriba se omite
* a ver la siguiente comilla doble, la bandera se baja
    
(nuevamente no se puede correr en jupyter porque ocupa un archivo para procesar 
 cuyo nombre se debe proporcionar como parámetro de linéa de instrucciones)

In [1]:
def procesa(line):
    salida = ''
    ignora = False
    for c in line:
        if c == '"':
            ignora = not ignora
            continue
        else:
            if c == '\n':
                return salida
            if not ignora:
                salida += c
            elif not c == ',':
                salida += c
    return salida

from sys import argv
with open(argv[1], 'r') as input:
    for line in input:
        print(procesa(line))

FileNotFoundError: [Errno 2] No such file or directory: '-f'

In [ ]:
mv ini.csv ini.raw
mv mc.csv mc.raw
mv ord.csv ord.raw
python3 puntuacion.py ini.raw > ini.csv
python3 puntuacion.py mc.raw > mc.csv
python3 puntuacion.py ord.raw > ord.csv
awk -F "," '{print NF}' ini.csv | sort | uniq -c
 878 19
awk -F "," '{print NF}' mc.csv | sort | uniq -c
 575 21
awk -F "," '{print NF}' ord.csv | sort | uniq -c
 459 22

Ya quedó, regresamos a pandas.

In [ ]:
>>> import pandas as pd
>>> enc_ini = pd.read_csv("ini.csv")
>>> enc_mcu = pd.read_csv("mc.csv")
>>> enc_ord = pd.read_csv("ord.csv")
>>> enc_ini.size
16663
>>> enc_mcu.size
12054
>>> enc_ord.size
10076

OK... ¿Dieciseismil qué es lo que tengo? Según el awk arriba, fueron 878 líneas y 19 columnas en la primera encuesta.

In [3]:
print(878 * 19)
print(877 * 19) # finalmente una celda que sí pueden correr en jupyter...

16682
16663


El segundo número coincide, por lo cual podemos concluir que pandas supo usar el primer renglón como una cabecera,
asignando nombres a los campos. Todo padre, pero qué se hace cuando el primer renglón *no* contiene una cabecera. Averiguemos.

In [ ]:
cat elisa/elisa.csv moi/moi.csv > profes.csv

head -n 3 profes.csv 
elisa,ej17,...6151,NP,NA
elisa,ej17,...5232,14,NA
elisa,ej17,...0180,75,NA

tail -n 3 profes.csv 
moi,ej18,...9511,100,NA
moi,ej18,...9547,100,NA
moi,ej18,...5244,NP,NA

wc -l profes.csv 
     521 profes.csv

Estos datos no contienen cabecera, pero sí son CSV de cinco columnas y 521 renglones.
Entonces esperamos obtener 2605 celdas.

In [ ]:
>>> cf = pd.read_csv("../profes.csv", header=None)
>>> cf.size
2605

Éxito. ¿Luego qué si no es un CSV sino tiene espacio como separador como le puse en elisa.dat?

In [ ]:
head -n 5 ../elisa/elisa.dat 
sem matr mc eo pr pf Hora_1ra T1_1ra T2_1ra T3_1ra T4_1ra T5_1ra Tareas_1ra PFE_1ra CF_1ra Derecho_1ra NPs_1ra Hora_2da T1_2da T2_2da T3_2da T4_2da T5_2da Tareas_2da PI_2da MC_2da Ord_2da PFE_2da CF_2da
ej17 ...6151 NA NA NA NA M4 NP NP NP NP NP 0 NP NP NC 8 NA NA NA NA NA NA NA NA NA NA NA NA
ej17 ...5232 NA NA NA NA M4 10 4 NP NP NP 14 NP 14 NC 6 NA NA NA NA NA NA NA NA NA NA NA NA
ej17 ...0180 6.0 12.5 5.0 1.0 M4 10 10 10 10 10 50 1 75 NA 0 NA NA NA NA NA NA NA NA NA NA NA NA
ej17 ...8175 0.0 9.0 NP NP M4 10 6 NP NP NP 16 NP 25 NC 4 NA NA NA NA NA NA NA NA NA NA NA NA

wc -l  ../elisa/elisa.dat 
     424 ../elisa/elisa.dat

awk '{print NF}' < ../elisa/elisa.dat | sort | uniq -c
  22 28
 402 29

Uh, oh... Hay que limpiar ese archivo antes de procesar. 22 renglones faltan una columna.
 
Inspeccionemos los archivos utilizados en P1 para crear elisa.dat en primer lugar, por si acaso:

In [ ]:
awk -F "," '{print NF}' < primera.csv | sort | uniq -c
 425 13
awk -F "," '{print NF}' < segunda.csv | sort | uniq -c
 102 14
awk '{print NF}' < resultados.txt  | sort | uniq -c
 359 6

Esos tres archivos todos contienen cosas coherentes.

In [ ]:
awk '{if (NF < 29) { print $1 } }' < ../elisa/elisa.dat | sort | uniq -c
   1 ad17
  21 ad18

Son los del último semestre más uno más viejo. A ver en emacs qué faltó agregar a esos. 
Curiosamente el archivo contiene exactamente 22 espacios dobles:

In [ ]:
python3 combine.py | grep "  " | wc -l 
      22

Obviamente sustituir el doble espacio auyda en que el formato esté consistente, pero no repone el dato faltante.

In [ ]:
python3 combine.py | grep "  " | sed 's/  / /g' | awk '{print NF}' | uniq -c
  22 28

Mejor veamos en qué posición falta el dato (usaré awk para ocultar las matrículas):

In [ ]:
iMac:elisa elisa$ python3 combine.py | grep "  " | sed 's/  / FALTA /g' | awk '{$2 = "..."; print $0}'
ad17 ... 2.0 10.0 NP NP V4 10 10 10 10 10 50 NP 62 FALTA 1 NA NA NA NA NA NA NA NA NA NA NA NA
ad18 ... 1.6666666666666665 7.75 7.0 3.1 M4 10 10 10 10 10 50 3.1 64 FALTA 0 M4 10 10 10 10 10 50 7 1.67 7.75 3.1 70
ad18 ... 6.0 4.5 NP NP M4 10 10 10 10 10 50 NP 54 FALTA 1 M4 10 10 10 10 10 50 NP 6 4.5 NP 61
ad18 ... 6.166666666666666 7.5 5.0 1.1 M4 10 10 10 10 10 50 1.1 65 FALTA 1 M4 10 10 10 10 10 50 5 6.17 7.5 1.1 70
ad18 ... 9.0 6.5 4.0 NP M4 10 10 10 10 10 50 NP 60 FALTA 1 M4 10 10 10 10 10 50 4 9 6.5 NP 70
ad18 ... 9.75 0.0 7.0 3.8 M4 10 10 10 10 10 50 3.8 64 FALTA 1 M4 10 10 10 10 10 50 7 9.75 0 3.8 71
ad18 ... 6.5 11.5 3.0 NP M4 10 10 10 10 10 50 NP 68 FALTA 1 M4 10 10 10 10 10 50 3 6.5 11.5 NP 71
ad18 ... 0.75 NP NP NP M4 10 10 10 10 10 50 NP 51 FALTA 2 M4 NP NP NP NP NP NP NP NP NP NP NP
ad18 ... 10.0 8.0 5.0 3.8 M4 10 10 10 10 10 50 3.8 63 FALTA 1 M4 10 10 10 10 10 50 5 10 8 3.8 77
ad18 ... 9.0 4.0 2.0 1.0 M4 10 10 10 10 10 50 1 58 FALTA 1 M4 10 10 10 10 10 50 2 9 4 1 66
ad18 ... 7.666666666666667 6.75 8.0 NP M4 10 10 10 10 10 50 NP 68 FALTA 0 M4 10 10 10 10 10 50 8 7.67 6.75 NP 72
ad18 ... 5.0 7.5 9.0 NP V1 10 10 10 10 10 50 NP 62 FALTA 0 V1 10 10 10 10 10 50 9 5 7.5 NP 72
ad18 ... 7.666666666666667 5.25 NP 3.9 V1 10 10 10 10 10 50 3.9 67 FALTA 1 V1 NP NP NP NP NP NP NP NP NP NP NP
ad18 ... 8.0 7.25 6.0 NP V1 10 10 10 10 10 50 NP 65 FALTA 1 V1 10 10 10 10 10 50 6 8 7.25 NP 71
ad18 ... 9.0 7.5 NP NP V1 10 6 10 10 10 46 NP 46 FALTA 1 V1 10 6 10 10 10 46 NP 9 7.5 NP 63
ad18 ... 8.0 11.5 5.0 2.1 V1 10 10 10 10 10 50 2.1 62 FALTA 1 V1 10 10 10 10 10 50 5 8 11.5 2.1 77
ad18 ... 6.0 10.25 5.0 2.2 V1 10 10 10 10 10 50 2.2 63 FALTA 1 V1 10 10 10 10 10 50 5 6 10.25 2.2 73
ad18 ... 0.75 10.5 9.0 NP V4 10 10 10 10 10 50 NP 62 FALTA 0 V4 10 10 10 10 10 50 9 0.75 10.5 NP 70
ad18 ... 12.0 10.75 NP 3.5 V4 10 10 10 10 10 50 3.5 69 FALTA 1 V4 10 10 10 10 10 50 NP 12 10.75 3.5 76
ad18 ... 9.0 9.5 NP NP V4 10 10 10 10 10 50 NP 68 FALTA 1 V4 10 10 10 10 10 50 NP 9 9.5 NP 69
ad18 ... 8.0 8.25 6.0 NP V4 10 10 10 10 10 50 NP 61 FALTA 1 V4 10 10 10 10 10 50 6 8 8.25 NP 72
ad18 ... 9.0 6.5 2.0 NP V4 10 10 10 10 10 50 NP 57 FALTA 1 V4 10 10 10 10 10 50 2 9 6.5 NP 68
iMac:elisa elisa$ head -n 1 elisa.dat
sem matr mc eo pr pf Hora_1ra T1_1ra T2_1ra T3_1ra T4_1ra T5_1ra Tareas_1ra PFE_1ra CF_1ra Derecho_1ra NPs_1ra Hora_2da T1_2da T2_2da T3_2da T4_2da T5_2da Tareas_2da PI_2da MC_2da Ord_2da PFE_2da CF_2da

Al parecer es el sí o no tienen derecho a segunda oportunidad que falta.

In [ ]:
awk '{if (NF == 29) {print $16}}' elisa.dat | sort | uniq -c
  91 CD
   1 Derecho_1ra
 192 NA
 118 NC

Debería decir CD (con derecho) o NC (no cumple) o NA (no aplica porque pasó en 1ra),
pero he olvidado mis propias reglas y no lo puse para ad18.
Efectivamente, abriendo primera.csv como hoja de cálculo en Numbers, contiene celdas sin valor.
Ordenando la columna en cuestión por su valor, es rápido rellenar lo que faltó.
Eran aquellos que sí tenían derecho en segunda oportunidad que tenían la celda vacía.

In [ ]:
python3 combine.py > elisa.dat

y ahora checamos otra vez:

In [ ]:
iMac:elisa elisa$ awk '{print NF}' < ../elisa/elisa.dat | sort | uniq -c
 424 29

Ya. Regresemos a pandas, entonces, esperando 423 * 29 = 12267 celdas, 
ya que sí hay cabecera, pero los separadores son espacios. 

In [ ]:
>>> elisa = pd.read_csv("../elisa/elisa.dat", sep=" ")
>>> elisa.size
12267

Ahora sigue la parte de combinar la información que tenemos por estudiante y semestre y profesor. 
Es importante recordar que un mismo alumno puede cursar simultáneamente con dos profesores 
(improbable, pero no imposible) y que un mismo alumno pueda cursar en múltiples semestres con un mismo profe
(en el caso que haya reprobado).

https://pandas.pydata.org/pandas-docs/stable/merging.html explica las formas de unir data frames. 

Todo va en función de cómo se llaman las columnas.
Mejor revisemos eso primero.

In [ ]:
>>> ' '.join(enc_ini.columns.values)
'sem tiempo grupo matr PE criterios semAnt significado hrsEstudio hrsComentario tecnicas califEsp baseEst prepa ingreso examenIng otrosEstudios mensaje promActual'
>>> ' '.join(enc_mcu.columns.values)
'sem tiempo grupo matr prom info cambio significado hrsEstudio comentarioHrs creditos corr tecnicas prepMC comoFue opinionMC califEsp baseEst cualTrabajo hrsTrabajo mensaje'
>>> ' '.join(enc_ord.columns.values)
'sem tiempo grupo matr cambio significado hrsEstudio hrsComentario tecnicas asesorias vecesAsesorias asesoriasComentario prepOrd comoVaya califEsp baseEst temas apoyo comoOrg medios temasGral mensaje'
>>> ' '.join(elisa.columns.values)
'sem matr mc eo pr pf Hora_1ra T1_1ra T2_1ra T3_1ra T4_1ra T5_1ra Tareas_1ra PFE_1ra CF_1ra Derecho_1ra NPs_1ra Hora_2da T1_2da T2_2da T3_2da T4_2da T5_2da Tareas_2da PI_2da MC_2da Ord_2da PFE_2da CF_2da'
>>> cf.columns = ['profe', 'sem', 'matr', '1ra', '2da']

Una bronca en lo que queremos lograr es que las encuestas no tienen un campo que indique el profesor, solamente uno que indica el grupo (un mismo alumno *no*, por suerte, puede inscribir más de un grupo con un mismo profe en un mismo semestre). Lo que se ocupa es *agregar* una columna profe en esos tres data frames, usando reglas tipo "si dice Elisa en el grupo, entonces ponle elisa como profe, pero si dice Moi, ponle moi" (no vamos a ni intentar comparar con Moisés; caracteres no-ASCII son la fuente de todas las penas). 

Como por el momento no contamos con resultados de otros profes (en fases futuras puede resultar necesario incluir más), dejemos los demás campos profe como NA/NaN/None o algo así que indica que aún no sabemos o aún no nos importa.

In [ ]:
enc_ini['profe'] = ['elisa' if 'Elisa' in str(valor) else 'moi' if 'Moi' in str(valor) else 'NA' for valor in enc_ini['grupo']]
enc_mcu['profe'] = ['elisa' if 'Elisa' in str(valor) else 'moi' if 'Moi' in str(valor) else 'NA' for valor in enc_mcu['grupo']]
enc_ord['profe'] = ['elisa' if 'Elisa' in str(valor) else 'moi' if 'Moi' in str(valor) else 'NA' for valor in enc_ord['grupo']]

Ahora sí podemos combinar esas tres tablas de encuestas con las calificaciones finales del data frame cf.

In [ ]:
>>> tmp = pd.merge(enc_ini, enc_mcu, how='outer', on=['profe', 'sem', 'matr'])
Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/reshape/merge.py", line 61, in merge
    validate=validate)
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/reshape/merge.py", line 555, in __init__
    self._maybe_coerce_merge_keys()
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/reshape/merge.py", line 986, in _maybe_coerce_merge_keys
    raise ValueError(msg)
ValueError: You are trying to merge on object and int64 columns. If you wish to proceed you should use pd.concat

Mmmmkay...

In [ ]:
>>> enc_ini.dtypes
sem              object
tiempo           object
grupo            object
matr             object
PE               object
criterios        object
semAnt           object
significado      object
hrsEstudio       object
hrsComentario    object
tecnicas         object
califEsp         object
baseEst          object
prepa            object
ingreso          object
examenIng        object
otrosEstudios    object
mensaje          object
promActual       object
profe            object
dtype: object
>>> enc_mcu.dtypes
sem              object
tiempo           object
grupo            object
matr              int64
prom             object
info             object
cambio           object
significado      object
hrsEstudio       object
comentarioHrs    object
creditos         object
corr             object
tecnicas         object
prepMC           object
comoFue          object
opinionMC        object
califEsp         object
baseEst          object
cualTrabajo      object
hrsTrabajo       object
mensaje          object
profe            object
dtype: object

Esta madre tiene razón. 
Por motivos misterio, en la segunda encuesta, las matrículas fueron interpretadas como enteros,
mientras en la primera encuesta, fueron strings (objetos). 

Suele pasar. Los obliguemos a strings, entonces.

In [ ]:
>>> enc_mcu['matr'] = [str(valor) for valor in enc_mcu['matr']]
>>> enc_mcu.dtypes
sem              object
tiempo           object
grupo            object
matr             object
prom             object
info             object
cambio           object
significado      object
hrsEstudio       object
comentarioHrs    object
creditos         object
corr             object
tecnicas         object
prepMC           object
comoFue          object
opinionMC        object
califEsp         object
baseEst          object
cualTrabajo      object
hrsTrabajo       object
mensaje          object
profe            object
dtype: object
>>> tmp = pd.merge(enc_ini, enc_mcu, how='outer', on=['profe', 'sem', 'matr'])
>>> tmp2 = pd.merge(tmp, enc_ord, how='outer', on=['profe', 'sem', 'matr'])
>>> datos = pd.merge(tmp2, cf, how='outer', on=['profe', 'sem', 'matr'])
Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/reshape/merge.py", line 61, in merge
    validate=validate)
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/reshape/merge.py", line 555, in __init__
    self._maybe_coerce_merge_keys()
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/reshape/merge.py", line 986, in _maybe_coerce_merge_keys
    raise ValueError(msg)
ValueError: You are trying to merge on object and int64 columns. If you wish to proceed you should use pd.concat
>>> cf.dtypes
profe    object
sem      object
matr      int64
1ra      object
2da      object
dtype: object
>>> cf['matr'] = [str(valor) for valor in cf['matr']]
>>> datos = pd.merge(tmp2, cf, how='outer', on=['profe', 'sem', 'matr'])
>>> datos.to_csv('datos.csv')

Bueno, ahora supuestamente está todo juntos en un nuevo archivo CSV. 
Aprovechemos la oportunidad para deshacernos de espacios blancos múltiple:

In [ ]:
grep '  ' datos.csv | wc -l 
     113
sed 's/  / /g' datos.csv > tmp
grep '  ' tmp | wc -l 
      10
sed 's/  / /g' tmp > tmp2
grep '  ' tmp2 | wc -l 
       0
mv tmp2 datos.csv

Regresemos al awk, sort y uniq para validar que todo esté como se quería y responder las preguntas planteadas.

In [ ]:
wc -l datos.csv 
    1023 datos.csv
awk -F "," '{print NF}' < datos.csv | sort | uniq -c
1023 62

So far, so good. Vamos a ver si quedó bien la columna indicando profesor:

In [ ]:
head -n 1 datos.csv | tr ',' '\012' | awk '{print NR": "$0}' | grep profe
21: profe

Lo puso en la columna 21. 

In [ ]:
awk -F "," '{print $21}' datos.csv | sort | uniq -c
 443 NA
 466 elisa
 113 moi
   1 profe

Son 466 registros de elisa, 113 de moi, una es la cabecera y 
hay 443 de profes que aún no se han identificado en el data frame.

In [ ]:
head -n 1 datos.csv | tr ',' '\012' | awk '{print NR": "$0}' | grep grupo
4: grupo_x
23: grupo_y
42: grupo

Comparemos con los grupos.

In [ ]:
awk -F "," '{print $21":\t"$4"\t"$23"\t"$42}' tmp | sort | uniq -c
   2 NA:			
   1 NA:			Blanca M6
   1 NA:			Sara
   1 NA:			Sara Jueves
   8 NA:			Sara Martes
   2 NA:		Blanca M2	Blanca M2
   1 NA:		Blanca M3	Blanca M3
   1 NA:		Blanca M4	Blanca M4
   2 NA:		Blanca M4 2102	
   1 NA:		Blanca M5	Blanca M5
   1 NA:		Blanca M5 2102	
   1 NA:		Blanca M6	
   3 NA:		Blanca M6	Blanca M6
   2 NA:		Blanca M6 2104	
   4 NA:		Sara Jueves	
   1 NA:		Sara Jueves	Sara
   2 NA:		Sara Jueves	Sara Jueves
   2 NA:		Sara Jueves	Sara Martes
   3 NA:		Sara Martes	
   1 NA:	Aldaco		
  15 NA:	Aldaco 7116		
   1 NA:	Aldaco 9201		
   2 NA:	Aldaco M1 1104		
  24 NA:	Aldaco M2 1104		
   1 NA:	Aldaco V1 1104		
   7 NA:	Blanca 2012		
  18 NA:	Blanca 2012	Blanca M4 2102	
  21 NA:	Blanca 2102 M3		
   5 NA:	Blanca 2102 M5		
  28 NA:	Blanca 2102 M5	Blanca M5 2102	
   7 NA:	Blanca 2104 M6		
  17 NA:	Blanca 2104 M6	Blanca M6 2104	
  23 NA:	Blanca 9104		
   1 NA:	Blanca M2 9104		
   1 NA:	Blanca M2 9104	Blanca M2	
  14 NA:	Blanca M2 9104	Blanca M2	Blanca M2
   2 NA:	Blanca M3 4105		
   1 NA:	Blanca M3 4105		Blanca M3
   4 NA:	Blanca M3 4105	Blanca M3	
   9 NA:	Blanca M3 4105	Blanca M3	Blanca M3
   2 NA:	Blanca M3 4105	Blanca M4	Blanca M4
   2 NA:	Blanca M4 2102		
   3 NA:	Blanca M4 2102	Blanca M4	
   7 NA:	Blanca M4 2102	Blanca M4	Blanca M4
   1 NA:	Blanca M5 2102		
   1 NA:	Blanca M5 2102	Blanca M4	Blanca M5
   1 NA:	Blanca M5 2102	Blanca M5	
   2 NA:	Blanca M5 2102	Blanca M5	Blanca M4
  20 NA:	Blanca M5 2102	Blanca M5	Blanca M5
   1 NA:	Blanca M6 2105		
   1 NA:	Blanca M6 2105		Blanca M6
   5 NA:	Blanca M6 2105	Blanca M6	
  25 NA:	Blanca M6 2105	Blanca M6	Blanca M6
   1 NA:	Celso		
   1 NA:	Sara		
   1 NA:	Sara	Sara	Sara Jueves
   1 NA:	Sara	Sara	Sara Martes
  20 NA:	Sara Jueves		
   1 NA:	Sara Jueves		Sara
   7 NA:	Sara Jueves		Sara Jueves
  10 NA:	Sara Jueves		Sara Martes
   5 NA:	Sara Jueves	Sara Jueves	
   8 NA:	Sara Jueves	Sara Jueves	Sara Jueves
  17 NA:	Sara Jueves	Sara Jueves	Sara Martes
  12 NA:	Sara Martes		
  11 NA:	Sara Martes		Sara Martes
   1 NA:	Sara Martes	Sara Jueves	Sara Martes
   3 NA:	Sara Martes	Sara Martes	
   3 NA:	Sara Martes	Sara Martes	Sara
   1 NA:	Sara Martes	Sara Martes	Sara Jueves
  28 NA:	Sara Martes	Sara Martes	Sara Martes
   9 elisa:			
   5 elisa:			Elisa V4
   2 elisa:		Elisa M4	Elisa M4
   1 elisa:		Elisa V1	
   2 elisa:		Elisa V1	Elisa V1
   3 elisa:		Elisa V4	
   1 elisa:		Elisa V4	Elisa M4
   1 elisa:	Elisa	Elisa M4	
  42 elisa:	Elisa M4		
   2 elisa:	Elisa M4		Elisa M4
  33 elisa:	Elisa M4	Elisa M4	
  70 elisa:	Elisa M4	Elisa M4	Elisa M4
   1 elisa:	Elisa M4	Elisa M4	Elisa V1
   2 elisa:	Elisa M4	Elisa M4	Elisa V4
   1 elisa:	Elisa M4	Elisa V4	
  39 elisa:	Elisa V1		
   1 elisa:	Elisa V1		Elisa M4
   6 elisa:	Elisa V1		Elisa V1
   1 elisa:	Elisa V1		Elisa V4
   2 elisa:	Elisa V1	Elisa M4	Elisa M4
   1 elisa:	Elisa V1	Elisa M4	Elisa V1
  24 elisa:	Elisa V1	Elisa V1	
  64 elisa:	Elisa V1	Elisa V1	Elisa V1
   1 elisa:	Elisa V1	Elisa V1	Elisa V4
   1 elisa:	Elisa V1	Elisa V4	Elisa V1
   2 elisa:	Elisa V1	Elisa V4	Elisa V4
  47 elisa:	Elisa V4		
   5 elisa:	Elisa V4		Elisa V4
   2 elisa:	Elisa V4	Elisa M4	Elisa V4
  25 elisa:	Elisa V4	Elisa V4	
  70 elisa:	Elisa V4	Elisa V4	Elisa V4
  54 moi:			
  13 moi:			Moises
   1 moi:			Moises 1101
   5 moi:		Moisés	
   8 moi:		Moisés	Moises
  19 moi:	Moises 1101		
   3 moi:	Moises 1101		Moises
   3 moi:	Moises 1101	Moisés	
   7 moi:	Moises 1101	Moisés	Moises
   1 profe:	grupo_x	grupo_y	grupo

OK, aquí ¿qué pasa? 

Los 54 de moi que no tienen grupo deben ser alumnos a los cuales Moisés capturó una calificación 
(están en el data frame cf) pero que no han contestado ninguna encuesta. 

Igual los 9 de elisa que no tienen grupo ninguno. Pinta que hicimos bien la asignación.

Regresemos a la metas planteadas para la P2:

1. combinar la información de los resultados de 1ra y 2da oportunidad con los datos de las encuestas

Ya quedó.

2. agregar una variable categórica para cada alumno, nombrando los casos de la siguiente forma
    * 1ra: aprobó la unidad de aprendizaje en primera oportunidad
    * 2da: aprobó la unidad de aprendizaje en segunda oportunidad
    * 3ra: presentó y reprobó ambas la 1ra y la 2da oportunidad

Esto lo podemos hacer ahora para alumnos de moi y elisa los dos. Regresemos a pandas brevemente.

In [ ]:
import pandas as pd
d = pd.read_csv("datos.csv")
casos = []
for i in d.index:
    cf1ra = str(d['1ra'][i])
    cf2da = str(d['2da'][i])
    if cf1ra.isdigit() and int(cf1ra) >= 70:
        casos.append('1ra')
    elif cf2da.isdigit() and int(cf2da) >= 70:
        casos.append('2da')
    elif cf1ra.isdigit() and cf2da.isdigit():
        casos.append('3ra')
    else:
        casos.append('NA') # con los datos actuales, no se sabe si tienen derecho                                               
d['caso'] = casos
d.to_csv('casos.csv')

In [ ]:
awk -F "," '{print $64}' < casos.csv | sort | uniq -c
 276 1ra
  52 2da
  18 3ra
 676 NA
   1 caso

Ahora queda la bronca de distinguir entre estos dos escenarios:

* 5ta: no participó lo suficiente para tener derecho a segunda oportunidad
*  NA: no se cuenta con información sobre sí o no participó o tuvo derecho en 1ra y 2da oportunidad 

lo que sí viene en elisa.dat: si dice NC, no tenía derecho a segundas. 

In [ ]:
grep NC elisa/elisa.dat | wc -l 
     118

3. extraer datos de encuestas para un caso específico o para una combinación de casos

In [ ]:
import pandas as pd
d = pd.read_csv("casos.csv")
theGood = d.loc[d['caso'] == '1ra']
theBad = d.loc[d['caso'] == '2da']
theUgly = d.loc[d['caso'] == '3ra']
theGood.to_csv("good.csv")
theBad.to_csv("bad.csv")
theUgly.to_csv("ugly.csv")

4. determinar cuántos estudiantes por semestre y por maestro hay en cada caso

In [ ]:
grep elisa good.csv | awk -F "," '{print $4}' | sort | uniq -c
  59 ad17
  60 ad18
  18 ej17
  66 ej18
grep moi good.csv | awk -F "," '{print $4}' | sort | uniq -c
  20 ad17
  23 ej17
  30 ej18
grep elisa bad.csv | awk -F "," '{print $4}' | sort | uniq -c
  12 ad17
   5 ad18
  17 ej17
  10 ej18
grep moi bad.csv | awk -F "," '{print $4}' | sort | uniq -c
   8 ej17
grep elisa ugly.csv | awk -F "," '{print $4}' | sort | uniq -c
   2 ad17
   3 ad18
   5 ej17
   8 ej18
grep moi ugly.csv | awk -F "," '{print $4}' | sort | uniq -c

Pues, ya fue un avance razonable para la segunda fase del proyecto. 

Con algo parecido con sus datos, ya liberan su reporte de la segunda práctica.

Prueben por lo menos la funcionalidad básica de pandas para

* leer data frames de archivos
* escribir data frames en CSV
* agregar columnas en data frames
* combinar dos o más data frames
* filtrar renglones de un data frame 

para convencerme que sí lo saben usar para eso.